In [1]:
#Импортирование необхожимых библиотек
import re
import os
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from tqdm import tqdm
from navec import Navec

In [2]:
#Класс для формирование последовательтей
class EmotionsDataset(data.Dataset):
    def __init__(self, train_true_path, test_false_path, emb_navec, batch_size=8):
        self.batch_size = batch_size
        self.emb_navec = emb_navec

        def text_cleaning(text_list):
            result_text = []
            for text in text_list:
                text = text.replace("\ufeff", "").lower().strip()
                text = re.sub(r'[^А-яA-z- ]', '', text)
                result_text.append(text.split())
            return result_text
                
        with open(train_true_path, "r", encoding="utf8") as file:
            self.train_true_text = file.readlines()
            self.train_true_text = text_cleaning(self.train_true_text)98I6

        with open(test_false_path, "r", encoding="utf8") as file:
            self.train_false_text = file.readlines()
            self.train_false_text = text_cleaning(self.train_false_text)

        self.data = list(map(lambda element: (element, 1.0), self.train_true_text)) + list(map(lambda element: (element, 0.0), self.train_false_text))
        self.data.sort(key=lambda element: len(element[0]))
        self.len_data = len(self.data)


    def __getitem__(self, item):
        item *= self.batch_size
        last_element = item + self.batch_size
        if last_element > self.len_data:
            last_element = self.len_data
            
        max_size = len(self.data[last_element-1][0])
        data_batch, target_batch = [], []
        data_tensors, target_tensors = [], []
        for i in range(item, last_element):
            data = []
            for index in range(max_size):
                if len(self.data[item][0]) - 1 >= index:
                    if self.data[i][0][index] in self.emb_navec:
                        data.append(torch.tensor(self.emb_navec[self.data[i][0][index]]).tolist())
                    else:
                        data.append(torch.zeros(300, dtype=torch.float32).tolist())
                else:
                    data.append(torch.zeros(300, dtype=torch.float32).tolist())
            data_tensors.append(data)
            target_tensors.append(self.data[i][1])
        data_tensors = torch.tensor(data_tensors)
        target_tensors = torch.tensor(target_tensors)

        return data_tensors, target_tensors

    def __len__(self):
        last_element = 0 if self.len_data % self.batch_size == 0 else 1
        return self.len_data // self.batch_size + last_element

In [3]:
#Сформируем модель нейронной сети
class EmotionsNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.layer = nn.Linear(2*hidden_size, 1)

    def forward(self, x):
        y, h = self.rnn(x)
        return self.layer(torch.cat([h[-2, :, :], h[-1, :, :]], dim=1))

In [10]:
import torch
import torch.nn as nn

batch_size = 3
seq_length = 17
d_size = 2

class GRUModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.gru = nn.GRU(2, 10, bias=False, batch_first=True)
        self.batchnorm = nn.BatchNorm1d(10)
        self.layer = nn.Linear(10, 4)
        
    def forward(self, x):
        xx = torch.zeros(size=(x.shape[0], seq_length, x.shape[-1]), dtype=torch.float32)
        xx[:, 0, :] = x
        y, h = self.gru(xx)
        pred = torch.empty(x.shape[0], seq_length, 4)
        for i in range(seq_length):
            pred[:, i, :] = self.layer(self.batchnorm(y[:, i, :]))
        return pred
            

# тензор x в программе не менять
x = torch.rand(batch_size, d_size)

model = GRUModel()
model.eval()
predict = model(x)
# здесь продолжайте программу

In [11]:
predict.shape

torch.Size([3, 17, 4])

In [5]:
#Формирование процесс обучения
eph = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

navec = Navec.load("navec_hudlit_v1_12B_500K_300d_100q.tar")
d_train = EmotionsDataset(train_true_path="train_data_true.txt", test_false_path="train_data_false.txt", emb_navec=navec)
train_data = data.DataLoader(d_train, batch_size=1, shuffle=True)

model = EmotionsNN(300, 16)
model.train()
model.to(device)

loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.0001)

for _e in range(eph):
    lm_count, loss_mean = 0, 0
    tqdm_train_data = tqdm(train_data, leave=True)
    for x_train, y_train in tqdm_train_data:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        predict = model(x_train.squeeze()).squeeze()
        loss = loss_func(predict, y_train.squeeze())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1 / lm_count * loss.item() + (1 - 1 / lm_count) * loss_mean
        tqdm_train_data.set_description(f"Текущая эпоха: [{_e}/{eph}]  Текущая ошибка: {round(loss_mean, 5)}")

Текущая эпоха: [99/100]  Текущая ошибка: 0.0005: 100%|█████████████████████████████████| 22/22 [00:00<00:00, 71.11it/s]


In [147]:
#Довольно хорошие предсказания, сохраним модель
state_dict = model.state_dict()
torch.save(state_dict, "model_emotions.tar")
print("Модель успешно сохранена!")

Модель успешно сохранена!


In [6]:
torch.cuda.is_available()

True

In [12]:
#Сделаем предсказание модели
model.to("cpu")
model.eval()
letters = "у меня все хорошо".split()